This is a starter notebook for the project, you'll have to import the libraries you'll need, you can find a list of the ones available in this workspace in the requirements.txt file in this workspace. 

In [ ]:

import os
import pandas as pd

# LangChain LLM and embeddings
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings

# Vector store (Chroma for RAG)
from langchain.vectorstores import Chroma

# Text splitting (if you break documents into chunks)
from langchain.text_splitter import CharacterTextSplitter

# Prompt templates (to format prompts for LLM)
from langchain.prompts import PromptTemplate

# Retrieval-based QA chain
from langchain.chains import RetrievalQA
import openai



In [ ]:
from langchain.llms import OpenAI
os.environ["OPENAI_API_KEY"] = "your-api-key"
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

# Generate 10 real estate listings using GPT

# Initialize LLM with turbo-instruct model
llm = OpenAI(
    model_name="gpt-3.5-turbo-instruct",
    temperature=0.7,
    max_tokens=400
)

# Prompt template for a single listing
listing_prompt = """
Generate a realistic real estate listing in the following format:

Neighborhood: [Name of neighborhood]
Price: [Price in USD]
Bedrooms: [Number of bedrooms]
Bathrooms: [Number of bathrooms]
House Size: [Square footage in sqft]

Description: [1-2 sentences describing the home and its key features]

Neighborhood Description: [Describe the neighborhood and amenities]
"""

# Generate 10 listings
listings = []
for i in range(10):
    response = llm(listing_prompt)
    listings.append(response)

# Preview first listing
print(listings[0])



Neighborhood: Northwood Park
Price: $450,000 USD
Bedrooms: 3
Bathrooms: 2.5
House Size: 2,200 sqft

Description: This charming colonial style home in Northwood Park boasts a spacious open layout with a modern kitchen, hardwood floors, and a cozy fireplace. The master suite features a walk-in closet and en-suite bathroom with a luxurious soaking tub.

Neighborhood Description: Northwood Park is a highly sought after neighborhood known for its quiet tree-lined streets and friendly community. Residents enjoy easy access to nearby parks, shopping centers, and top-rated schools. The neighborhood also offers a community pool and clubhouse for residents to relax and socialize.


In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

# Create FAISS index
from langchain.indexes import VectorstoreIndexCreator
from langchain.document_loaders import TextLoader
from langchain.schema import Document

# Save listings to a file
with open("listings.txt", "w") as f:
    for listing in listings:
        f.write(listing + "\n\n")

loader = TextLoader("listings.txt")
index = VectorstoreIndexCreator().from_loaders([loader])

# Setup ChatOpenAI (chat model)
chat_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Create retriever from FAISS index
retriever = index.vectorstore.as_retriever(search_kwargs={"k": 5})

# Create RetrievalQA chain
rag_chain = RetrievalQA.from_chain_type(
    llm=chat_llm,
    chain_type="stuff",
    retriever=retriever
)

# Query
query = "Find a house with 3 bedrooms and a large backyard in a quiet neighborhood."
result = rag_chain.run(query)

print(result)



Based on the information provided, the house that fits your criteria is the one in Parkside Heights. It has 3 bedrooms, a spacious backyard with a patio, and is located in a highly sought-after neighborhood known for its family-friendly atmosphere.


In [ ]:
# Collect Buyer Preferences

# Define the questions
questions = [   
    "How big do you want your house to be?", 
    "What are 3 most important things for you in choosing this property?", 
    "Which amenities would you like?", 
    "Which transportation options are important to you?",
    "How urban do you want your neighborhood to be?"
]

# Simulate user responses (hardcoded for now)
answers = [
    "A comfortable three-bedroom house with a spacious kitchen and a cozy living room.",
    "A quiet neighborhood, good local schools, and convenient shopping options.",
    "A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.",
    "Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.",
    "A balance between suburban tranquility and access to urban amenities like restaurants and theaters."
]

# Combine answers into a single preference text
buyer_preferences = " ".join(answers)

print("Buyer Preferences Summary:\n", buyer_preferences)


Buyer Preferences Summary:
 A comfortable three-bedroom house with a spacious kitchen and a cozy living room. A quiet neighborhood, good local schools, and convenient shopping options. A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system. Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads. A balance between suburban tranquility and access to urban amenities like restaurants and theaters.


In [14]:
# Query FAISS index using preferences
result = rag_chain.run(buyer_preferences)
print("Top Match for Buyer:\n", result)


Top Match for Buyer:
 Based on the information provided, the neighborhood that seems to best match your criteria is Parkside Heights. It offers a quiet and family-friendly atmosphere, convenient access to major highways and public transportation, nearby community parks, and a variety of local shops and restaurants. The houses in this neighborhood have modern amenities like updated kitchens, spacious layouts, and backyard spaces perfect for gardening and entertaining. However, specific details about a two-car garage and energy-efficient heating systems are not mentioned in the descriptions provided.


In [ ]:
# Personalize the listing description

# Define personalization prompt 
personalize_prompt = f"""
You are a real estate agent. Personalize the following property listing
to highlight the features most relevant to this buyer's preferences.

Buyer Preferences:
{buyer_preferences}

Property Listing:
{result}

Make sure to emphasize matching features but do not add new details not in the listing.
"""

personalized_listing = llm(personalize_prompt)

print("Personalized Listing:\n")
print(personalized_listing)



Personalized Listing:


Welcome to your dream home in Parkside Heights! This charming three-bedroom house boasts a spacious kitchen, perfect for cooking and entertaining. The cozy living room offers a comfortable space to relax and unwind after a long day. Located in a quiet and family-friendly neighborhood, this home is just minutes away from top-rated local schools and convenient shopping options.

Step outside into your own backyard oasis, perfect for gardening and outdoor activities. And with a two-car garage, you'll have plenty of space for storage and parking. The modern and energy-efficient heating system will keep you warm and cozy during the colder months.

But that's not all, this home also offers easy access to a reliable bus line, making your commute a breeze. Plus, with proximity to a major highway and bike-friendly roads, you'll have easy access to both suburban tranquility and urban amenities like restaurants and theaters.

Don't miss out on the opportunity to make this 